In [14]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
import tensorflow.keras as keras
from tensorflow.keras import callbacks
from tensorflow.keras import layers
import cv2 as cv
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelBinarizer
import numpy as np

from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw

import glob
import random


In [15]:
def sharpen_image(image):
    image_median_blurred = cv.medianBlur(image, 3)
    image_gaussian_blurred = cv.GaussianBlur(image, (0, 0), 3)
    image_sharpened = cv.addWeighted(image_median_blurred, 1.2, image_gaussian_blurred, -0.8, 0)
    _, thresh = cv.threshold(image_sharpened, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    # thresh = cv.adaptiveThreshold(image_sharpened, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 15, 8)

    kernel = np.ones((1, 1), np.uint8)
    thresh = cv.erode(thresh, kernel)

    return thresh

In [ ]:
((train_data_vanilla, train_label), (test_data_vanilla, test_label)) = mnist.load_data()

train_data_vanilla = np.array(list(map(sharpen_image, train_data_vanilla)))
cv.imshow("",train_data_vanilla[0])
cv.waitKey(0)


In [ ]:
train_data = train_data_vanilla.flatten().reshape(train_data_vanilla.shape[0], np.prod(train_data_vanilla.shape[1:]))
test_data = test_data_vanilla.flatten().reshape(test_data_vanilla.shape[0], np.prod(test_data_vanilla.shape[1:]))


#preprocessor_CNN = RobustScaler()
preprocessor_CNN = MinMaxScaler()
#preprocessor_CNN = StandardScaler()
train_data_CNN = preprocessor_CNN.fit_transform(train_data).reshape((train_data.shape[0],28 ,28, 1))
test_data_CNN = preprocessor_CNN.transform(test_data).reshape((test_data.shape[0], 28,28, 1))

preprocessor_label = LabelBinarizer()
train_label_CNN = preprocessor_label.fit_transform(train_label)
test_label_CNN = preprocessor_label.transform(test_label)



In [8]:
neural_model = Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(32, kernel_size=5, padding="same", activation="relu"),
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.Dropout(0.2),
    layers.Conv2D(64, kernel_size=3, activation="relu"),
    layers.MaxPooling2D(pool_size=2, strides=2),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation="softmax")
])
early_stopping = callbacks.EarlyStopping(
    min_delta=0.01,
    patience=5,
    restore_best_weights=True
)
adam = keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-7,
    amsgrad=False,
    name="Adam"
)
neural_model.compile(
    optimizer = adam,
    loss = "categorical_crossentropy",
    metrics=['accuracy']
)

In [5]:
result = neural_model.fit(
    train_data_CNN,
    train_label_CNN,
    batch_size = 128,
    validation_data=(test_data_CNN, test_label_CNN),
    epochs = 10,
    #shuffle=True,
    #verbose=False,
    use_multiprocessing=True,
    callbacks=[early_stopping]
)

2021-12-02 14:59:58.116066: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/100
129/469 [=======>......................] - ETA: 18s - loss: 0.5728 - accuracy: 0.8231 

KeyboardInterrupt: 